In [1]:
# Soroban Flash Anzan
### Versión de prueba



In [63]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

from soroban_operations_generator import SorobanOperationGenerator
import time
import pandas as pd
import numpy as np



In [ ]:
class SorobanFlashAnzan:
    def __init__(self):
        self.score = 0
        self.total_questions = 0
        self.current_answer = None

        self.question_label = widgets.Label(value="Presiona 'Iniciar' para comenzar.")
        self.operation_table = widgets.Output()

        self.step_input = widgets.Dropdown(options=SorobanOperationGenerator.pasos, value='Paso 5', description='Paso:')
        self.digits_input = widgets.IntText(value=2, description='Dígitos:')
        self.len_operations_input = widgets.IntText(value=5, description='Longitud Operaciones:')
        self.num_questions_input = widgets.IntText(value=5, description='Número de Preguntas:')
        self.show_all_questions = widgets.Checkbox(value = False, description = 'Mostrar todas las Operaciones')

        

        self.answer_input = widgets.Text(placeholder='Escribe tu respuesta aquí')
        self.submit_button = widgets.Button(description='Enviar')
        self.start_button = widgets.Button(description='Iniciar')
        self.result_label = widgets.Label(value="")

        self.submit_button.on_click(self.check_answer)
        self.answer_input.continuous_update = False
        self.answer_input.observe(self._on_enter, names='value')
        self.answer_input.disabled = True
        self.submit_button.disabled = True
        self.start_button.on_click(self.start_quiz)
        input_widgets = widgets.VBox([
            self.step_input,
            self.digits_input,
            self.len_operations_input,
            self.num_questions_input,
            self.show_all_questions,
            self.question_label,
            self.answer_input,
            self.start_button,
            self.submit_button,
            self.result_label
            
        ])
        display(widgets.HBox([input_widgets, self.operation_table]))



    def _on_enter(self, change):
        if change['type'] == 'change' and change['name'] == 'value' and change['new'] != '':
            self.check_answer(None)

    def start_quiz(self, b):
        self.score = 0
        self.total_questions = 0
        self.answers = []
        self.start_button.disabled = True
        self.df1 = pd.DataFrame(index = list(
            range(1, self.len_operations_input.value+1)) + ['Total'])
        self.next_question()

    def next_question(self):
        self.operation_generator = SorobanOperationGenerator(step=self.step_input.value, digits=self.digits_input.value)
        ops, sum_complete = self.operation_generator.get_list_of_operations(self.len_operations_input.value-1)
        self.current_answer = self.operation_generator.get_int_operation(sum_complete)
        self.answers.append(self.current_answer)
        self.answer_input.disabled = False
        self.submit_button.disabled = False
        self.answer_input.value = ""
        self.result_label.value = ""
        # Display operation in table
        with self.operation_table:
            from IPython.display import clear_output, display
            clear_output()
            df = pd.DataFrame([self.operation_generator.get_int_operation(_) for _ in ops ], columns=[self.total_questions + 1], 
                              index = list(range(1,self.len_operations_input.value+1)))

            df1 = pd.concat([self.df1, df], axis = 1)

            self.df1 = df1.copy()
            styled = self._style_df1(self.df1)
            styled.set_table_styles([
                {'selector': 'th, td', 'props': [('font-size', '20px'), ('padding', '10px')]},
                {'selector': 'table', 'props': [('width', '100%'), ('min-width', '600px')]}])
            display(styled)
        

    def _style_df1(self, df):
        def highlight_total(val, col):
            try:
                idx = int(col) - 1  # columns are question numbers (1-based)
                if idx < len(self.answers) and str(val) == str(self.answers[idx]):
                    return 'background-color: lightgreen; color: black;'
                elif np.isnan(val):
                    return ''

                else:
                    return 'background-color: salmon; color: black;'
            except Exception:
                return ''
        def safe_format(val):
                try:
                    return "{:.0f}".format(float(val))
                except Exception:
                    return str(val)
                
        if 'Total' in df.index:
            # Remove any text, show only the int value, and apply color
            
            styled = df.style.format(safe_format)
            styled.set_table_styles([
                {'selector': 'th, td', 'props': [('font-size', '20px'), ('padding', '10px')]},
                {'selector': 'table', 'props': [('width', '100%'), ('min-width', '600px')]}])
            
            return styled.apply(lambda row: [highlight_total(row[col], col) if row.name == 'Total' else '' for col in df.columns], axis=1)
        else:
            styled = df.style.format(safe_format)
            styled.set_table_styles([
                {'selector': 'th, td', 'props': [('font-size', '20px'), ('padding', '10px')]},
                {'selector': 'table', 'props': [('width', '100%'), ('min-width', '600px')]}])
            return styled

    def check_answer(self, b):
        user_answer = self.answer_input.value

        if user_answer.isdigit() and int(user_answer) == self.current_answer:
            self.score += 1
            self.result_label.value = 'Tu respuesta es correcta!'
        else:
            self.result_label.value = f'Tu respuesta es incorrecta. La respuesta correcta era {self.current_answer}.'
        
        self.total_questions += 1
        if self.total_questions >= self.num_questions_input.value:
            self.df1.at['Total', self.total_questions] = user_answer
            with self.operation_table:
                from IPython.display import clear_output, display
                clear_output()
                display(self._style_df1(self.df1))
            self.question_label.value = f"Quiz terminado. Tu puntuación: {self.score}/{self.total_questions}"
            self.result_label.value = ""
            self.start_button.disabled = False
            self.answer_input.value = ""
            self.answer_input.disabled = True
            self.submit_button.disabled = True
            

        else:
            self.question_label.value = f'Número de preguntas resueltas: {self.total_questions}/{self.num_questions_input.value}'
            self.df1.at['Total', self.total_questions] = user_answer
            with self.operation_table:
                from IPython.display import clear_output, display
                clear_output()
                display(self._style_df1(self.df1))
            time.sleep(2)
            self.next_question()
    

In [148]:
SorobanFlashAnzan()